In [2]:
sc

In [3]:
# Reading parquet file

df = spark.read.format('parquet').load('abfss://refined@projectsynapsestorage.dfs.core.windows.net/ExtNew/E69C5259-86B9-4B9F-85DA-AB181EF11BD0_47_0-1.parquet')

In [4]:
display(df)

In [5]:
# Get all columns into a variable

AllColumns = df.schema.names

print(AllColumns)

In [6]:
for column in AllColumns:
    nullCount = df.filter(df[column].isNull()).count()
    if(nullCount > 0):
        print('Null Count in '+ column + ' - ' + str(nullCount))

In [7]:
# Group by aggregation
# To know total employed citizens based on gender

print('To know total employed citizens based on gender')
df.groupBy('Gender').sum('Employed').show()

#2 Applying multiply aggregtion susing agg()

print('Applying multiply aggregtion susing agg()')
from pyspark.sql.functions import sum,avg
df.groupBy('Gender').agg(sum('Employed'), avg('Employed')).show()

In [8]:
# Find duplicate row count using group by
from pyspark.sql.functions import col

print('Find duplicate row count using group by')

df.groupBy('Line Number','Year').count().filter(col('count')>1).show()

In [9]:
df.filter((col('Line Number').isNull() & (col('Year').isNull()))).show()

In [10]:
# Using Drop duplicates to handle duplicate rows

print('# Using Drop duplicates to handle duplicate rows')

df_nodups = df.drop_duplicates()

In [11]:
df_nodups.groupBy('Line Number','Year').count().filter(col('count')>1).show()

In [12]:
df_nodups.filter((col('Line Number').isNull() & (col('Year').isNull()))).show()

In [13]:
# Using Dropna 
print('Using Dropna ')

df_all = df_nodups.dropna(how='all')

In [14]:
df_all.filter((col('Line Number').isNull() & (col('Year').isNull()))).show()

In [15]:
for column in AllColumns:
    nullCount = df_all.filter(df[column].isNull()).count()
    if(nullCount > 0):
        print('Null Count in '+ column + ' - ' + str(nullCount))

In [16]:
# Replacing the NUL values with unknown

print('Replacing the NUL values with unknown FOR ALL COLUMNS')

df_clean = df_all.fillna('unknown')

In [17]:
for column in AllColumns:
    nullCount = df_clean.filter(df_clean[column].isNull()).count()
    print('Null Count in '+ column + ' - ' + str(nullCount))

In [18]:
# Writing the data to a location 

df_clean.write.format('parquet')\
             .mode('overwrite')\
             .save('abfss://refined@projectsynapsestorage.dfs.core.windows.net/NoNulls/')